# Purpose
This report presents the analysis of the venues in London which are equipped with a wheelchair ramp. </p>
A dataset with ~4000 observations was used, mapping different venue attributes to the binary variable of having a wheelchair ramp or not. </p>
A model was then developed to predict whether a venue already has a wheelchair ramp. The prediction accuracy is ~65%. True-negative-rate > 66%. </p>

# Motivation
The marketing department aims at expanding the potential market for wheelchair installations to event venues. </p>
Reportedly, 40 % of event venues already have a wheelchair ramp. </p>
Due to reputation concerns and resource restrictions, it is unfeasible to query every venue with regards to the existence of a ramp.</p>
Therefore, aiming to optimize the lead generation process, an approriate model has to be developed to predict whether or not a venue already has a ramp.

### Preliminary Remarks
The terms _variable_ and _feature_ are used interchangeably. </p>
</p>
The following independent variables are given
<ul>
  <li>venue_name : The name of the venue</li>
  <li>Loud music / events : whether the venue hosts loud events (True) or
not (False)</li>
  <li>Wi-Fi : Numeric, whether the venue provides alcohol (1) or not (0)</li>
  <li>supervenue : Character, whether the venue qualifies as a supervenue
(True) or not (False).</li>
  <li>max_standing :  Numeric, the total standing capacity of the venue.</li>
  <li>Theatre_max : Numeric, the total capacity of the theatre.</li>
  <li>Promoted / ticketed events : Character, whether the venue hosts promoted/ticket
events (True) or not (False).</li>
</ul></p>
The following dependent variable is given
<ul>
    <li>Wheelchair accessible : Character, whether the venue is wheelchair accessible
        (True) or not (False).</li>
</ul>

### Analysis Structure
The report is structured in three parts:</p>
An __EDA__, description of the __Model Development__ and a __Conclusion__.</p>
Special emphasis is put on reproducibility and explainability of the result derivation.</p>

# Exploratory Data Analysis

In [ ]:
# Use this cell to begin, and add as many cells as you need to complete your analysis!
# Perform necessary imports and installs
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

!pip install pandas-profiling
!pip install optuna
!pip install lime

import numpy as np
import pandas as pd

import phik
from phik.report import plot_correlation_matrix
from phik import report

import matplotlib.pyplot as plt
import seaborn as sns
clear_output()

     |████████████████████████████████| 262 kB 28.4 MB/s            
     |████████████████████████████████| 12.6 MB 2.4 MB/s            
     |████████████████████████████████| 102 kB 20.0 MB/s           
     |████████████████████████████████| 292 kB 104.1 MB/s            
     |████████████████████████████████| 696 kB 104.7 MB/s            
     |████████████████████████████████| 11.7 MB 101.1 MB/s            
     |████████████████████████████████| 78 kB 19.2 MB/s            
     |████████████████████████████████| 133 kB 113.2 MB/s            
     |████████████████████████████████| 16.8 MB 76.9 MB/s            
     |████████████████████████████████| 63 kB 3.6 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 306 kB 91.0 MB/s            
     |████████████████████████████████| 4.7 MB 96.2 MB/s            
     |████████████████████████████████| 41.6 MB 92.2 MB/s            
     |████████████████████████████████| 11.3 MB 88.5 MB/s 

First, the variables are checked for dtypes. __Alcohol__ is converted to dtype _bool_.</p>
The variable names are adjusted for better consistency.</p>
The dataset is checked for missing values (none).

In [ ]:
# Import data into Dataframe and describe numerical data
venues = pd.read_csv("data/event_venues.csv")
venues.dtypes

In [ ]:
# Rename columns for consistency and clarity
venues = venues.rename(columns = {
    "venue_name":"Venue",
    "Loud music / events": "Loud Music",
    "Venue provides alcohol": "Alcohol",
    "supervenue": "Supervenue",
    "U-Shaped_max":"U Cap",
    "max_standing":"Standing Cap",
    "Theatre_max":"Max Cap",
    "Promoted / ticketed events": "Promo Events",
    "Wheelchair accessible":"WC Ramp",
})

In [ ]:
# Change "Alcohol" to binary type
venues["Alcohol"] = venues["Alcohol"].astype(bool)

# Change the remaining numeric variables to dtype = int. (Capacity of humans in float doesn't make sense)
venues[["U Cap", "Standing Cap", "Max Cap"]] = venues[["U Cap", "Standing Cap", "Max Cap"]].astype(int)

In [ ]:
# Check for na values
venues.isnull().any()

In [ ]:
# Check for duplicates
duplicates = venues.duplicated().sum()
uniques = len(pd.unique(venues.Venue))
print(f"Duplicate observations: {duplicates} | Unique venues: {uniques}")

In [ ]:
# Sort dataframe
# Keep only the first entry for each duplicate, drop the rest
total_ven_ramp = venues["WC Ramp"].sum()/len(venues)
venues = venues.sort_values(by = "Venue").drop_duplicates()
print(f"{total_ven_ramp:.2f} % of events had ramps.")

## Dataset Summary
<ul>
    <li>The dataset has 3910 observations. </li> 
    <li>450 (11.51 %) are duplicates. </li>
    <li>1444 (36.93 %) venues are unique. </li>
</ul>

The same venue can have different capacities and attributes. For example, one venue can allow alcohol on one occasion and prohibit it on another. Therefore, there are less unique venues than unique observations in the dataset. </p>
__3__ variables are __numeric__ and real-valued: "U Cap", "Standing Cap" and "Max Cap", describing the capacities of the venue </p>
__6__ are __binary__: "Loud Music", "Alcohol", "Wi-Fi", "Supervenue", "Promo Events", "WC Ramp" </p>
Dependent variable: __"WC Ramp"__ (specifies whether a wheelchair ramp is available)

In [ ]:
# Count how many venues vary with respect to having a wheelchair ramp, how many always have it, how many never have it
check_fractions = venues.groupby("Venue")["WC Ramp"].sum()/venues.groupby("Venue")["WC Ramp"].count()
ven_var_ramp = ((check_fractions != 0.0) & (check_fractions != 1.0)).sum()
ven_with_ramp = ((check_fractions == 1.0)).sum()
ven_wo_ramp = ((check_fractions == 0.0)).sum()

assert ven_var_ramp + ven_with_ramp + ven_wo_ramp == len(check_fractions), "Ratios must sum to 1"

#print(f"{ven_var_ramp} venues ({ven_var_ramp/len(check_fractions):.2f} %) are inconsistent with regards to providing a wheelchair ramp.")
#print(f"{ven_with_ramp} venues ({ven_with_ramp/len(check_fractions):.2f} %) always have a ramp.")
#print(f"{ven_wo_ramp} venues ({ven_wo_ramp/len(check_fractions):.2f} %) never have a ramp.")

In [ ]:
# Calculate the fraction of unique venues that have a wheelchair ramp (including those with inconsistent use)
unique_venues = venues.drop_duplicates(subset=["Venue", "WC Ramp"])
weelchair_ratio = unique_venues["WC Ramp"].sum()/len(unique_venues)
#print(f"{weelchair_ratio*100:.2f} % of venues have a wheelchair ramp")

## How many venues have ramps?
In summary:</p>
Of the unique venues, __38 %__ __always__ have a ramp. </p>
__46 %__ of the venues __never__ have a ramp. </p>
__16 %__ are __inconsistent__ in their use. </p>
__50 %__ of the individual __events__ had ramps. </p>

In [ ]:
# Plot ramp frequencies
sns.set(font_scale = 1, style = "white")
fig, ax = plt.subplots(figsize = (6, 5))
ramp_frac = pd.DataFrame([ven_wo_ramp/len(check_fractions), ven_with_ramp/len(check_fractions),  ven_var_ramp/len(check_fractions)])
ramp_frac = ramp_frac.rename(index = {0:"Without Ramp", 1:"With Ramp", 2:"Inconsistent"}, columns = {0:"Frequency"})
sns.barplot(y = ramp_frac.index, x = "Frequency", data = ramp_frac, orient = "h", ax = ax)
ax.set_title("46 % of venues don't have a ramp.\n16 % report inconsistent use.", pad = 20)
plt.xlim(0, 1)

Duplicate observations have been removed from the dataset.  
Venues that have inconsistent entries with regards to having a ramp are kept.  
More information on the causation of the inconsistent entries would be necessary to decide which observations to keep and which to drop.  
One hypothesis could be that the inconsistent venues didn't have a ramp at first, but subsequently added it, or that temporary ramps were available for certain events.

## Distributions of variables
The distribution of the numerical and binary variables is explored more in-depth to understand the dataset.  
Relationships between the independent variables and/or the dependent variable are investigated to prepare the model development and identify related variables.</p>
The __goal__ is to understand __which variables indicate__ that a ramp is __already installed__ at a venue.</p>

### Numeric variables

The pandas description of the numeric variables in the dataset shows that the capacity of the venues varies greatly. </p>
For the variable __Standing Cap__, the __mean__ value (112.9) is more than __double__ the __median__ value (60.0)

In [ ]:
# Summary description of numeric variables
numericals = venues.select_dtypes(include = "number")
numericals.describe(include = "all")

In [ ]:
# Get median 
numericals.median()

Investigation of the __ECDF__ plots of the numerical variables suggests __exponential distributions__, with most values in the lower range and a few outliers.

In [ ]:
# Plot ECDFs
fig, axs = plt.subplots(1, 3, sharey = True, figsize = (10, 5))
for i, col in enumerate(numericals.columns):
	sns.ecdfplot(venues[col], ax = axs[i])
	axs[i].set_xlabel(col + " [-]")

axs[0].set_ylabel("ECDF [%]")
fig.suptitle("The numeric variables are exponentially distributed")
fig.tight_layout()

Analysis of the box plots confirms this view. The outliers of the boxplot below have been omitted to increase visibility.

In [ ]:
numericals["WC Ramp"] = venues["WC Ramp"]
numericals_melted = pd.melt(numericals, id_vars = "WC Ramp", var_name = "Cap Type", value_name = "Capacity").sort_values("WC Ramp", ascending = False)

ax = sns.boxplot(x = "Cap Type", y = "Capacity", hue = "WC Ramp", data = numericals_melted, showfliers = False)
ax.set_title(f"Venues with a ramp have a higher variance of capacity. \n Venues with a ramp seem to have higher standing capacity.")
ax.set_ylabel("Capacity [-]")
ax.set_xlabel("Capacity Type")

It is odd, that __U Cap__ shows almost not variance.  
Therefore the values in __U Cap__ are counted, showing that in __73 %__ of the cases, the __U Cap__ value is __35__. The next most frequent value is 30 with only 3 % frequency.

In [ ]:
venues["U Cap"].value_counts()/len(venues)

A possible hypothesis for this could be that part of the dataset was derived from an online form, where 35 could be specified as the lowest U-Cap number.  
Another possibility is that the dataset is corrupted. Since this is not provable from the given information, the data is not changed.

We further check for consistency with __Max Cap__.  
__U Cap__ must be strictly smaller than __Max Cap__.

In [ ]:
u_larger_max = venues["Max Cap"] < venues["U Cap"]
print(f"In {u_larger_max.sum()} cases, the given U-Cap value is larger than the Max Cap value")
venues.loc[u_larger_max, "U Cap"] = venues.loc[u_larger_max, "Max Cap"]

192 observations don't satisfy the requirement. The __U Cap__ value for those observations is set to the value of __Max Cap__.

From plotting the single variables in a jointplot (not shown here), we can observe that __large venues__ have a ramp in most cases and define a large venue as: </p>
U-Shape Capacity > 200  
Standing Capacity > 500  
Maximum Capacity > 200  

To test whether the means between large and small venues are different, the columns are split at the given thresholds.

In [ ]:
# Get means for large and small events
means = pd.DataFrame(columns = numericals.drop("WC Ramp", axis = 1).columns)
thresh = [200, 500, 200]

# Store means for large and small events and get diff of means
for i, col in enumerate(means.columns):
	means.loc[0, col] = round(numericals.loc[numericals[col] > thresh[i], "WC Ramp"].mean(), 2)
	means.loc[1, col] = round(numericals.loc[numericals[col] <= thresh[i], "WC Ramp"].mean(), 2)
	means.loc[2, col] = means.loc[0, col] - means.loc[1, col]
means["Venue Size"] = ["large", "small", "diff"]
means = means.set_index("Venue Size")

The mean probability of having a  ramp at a large venue is almost double as high as at smaller venues.</p>
The mean differences can be seen below.

In [ ]:
means

In [ ]:
# Define function to test for difference in test statistics
def bootstrap_diff_metric(data_1, data_2, func, diff_value, draws = 10000) -> [list, float, list]:
    """ 
    This function trains the given classifiers.
    Args:
        data_1: array_like: the first array with data to test. 
        data_2: array_like: the second array, being subtracted from func(data_1)
        func: function: test_metric with signature func(data_1, data_2) and single-value return
        diff_value: int: the test statistic from the empirical data to be tested against
        draws: int (Default = 10000): number of random draws
    Returns:
        output: [list, float, list]: returns a tuple of the confidence intervals, the p_value and the samples
    """      
    samples = []
    data = np.concatenate((data_1, data_2))
    overall_mean = data.mean()
    data_1 = data_1 - data_1.mean() + overall_mean
    data_2 = data_2 - data_2.mean() + overall_mean
    for _ in range(draws):
        diff_sample = func(np.random.choice(data_1, size = len(data_1))) - func(np.random.choice(data_2, size = len(data_2)))
        samples.append(diff_sample)
    p_value = np.sum(np.array(samples) >= diff_value)/draws
    ci = np.percentile(samples, [2.5, 97.5])
    return ci, p_value, samples

The hypothesis that __large venues__ have more wheelchair ramps than small venues was checked next.</p>
__H0: mean(large) - mean(small) = 0__</p>
For this, the datasets for large and small venues were merged and the overall mean calculated. The datasets were then equalized to have the same mean.</p>
The bootstraps are drawn __pseudo-randomly__ from the merged and equalized dataset.  
A total of __10,000 bootstraps__  each was performed.  
The __p-value__ and __95 % confidence-intervals__ are returned by the function defined above.

In [ ]:
# Calculate p-value and ci for U-Cap
ucap_ci, ucap_p, ucap_bs = bootstrap_diff_metric(
    venues.loc[venues["U Cap"] > 200, "WC Ramp"], 
    venues.loc[venues["U Cap"] <= 200, "WC Ramp"], 
    np.mean, 
    means.iloc[2, 0])
ucap_p

In [ ]:
# Calculate p-value and ci for Standing Cap
stcap_ci, stcap_p, stcap_bs = bootstrap_diff_metric(
    venues.loc[venues["Standing Cap"] > 1000, "WC Ramp"], 
    venues.loc[venues["Standing Cap"] <= 1000, "WC Ramp"], 
    np.mean, 
    means.iloc[2, 1])
stcap_p

In [ ]:
# Calculate p-value and ci for Standing Cap
maxcap_ci, maxcap_p, maxcap_bs = bootstrap_diff_metric(
    venues.loc[venues["Max Cap"] > 700, "WC Ramp"], 
    venues.loc[venues["Max Cap"] <= 700, "WC Ramp"], 
    np.mean, 
    means.iloc[2, 2])
maxcap_p

__p-values__</p>
The difference is significant in all for __Standing Cap__ and __Max Cap__, with p-values (might vary slightly):</p>
<ul>
    <li>U Cap: 0.06</li>
    <li>Standing Cap: 0.0007</li>
    <li>Max Cap: 0.0</li>
</ul>
U-Cap is slightly not significant (5 % limit), but consideration of the effect should not be discarded.</p>
We can conclude that large venues (as defined above) are relatively more likely to have wheelchair ramps than smaller venues.

### Additional Features

The binary variable __Large Venue__ is therefore introduced into the __venues__ dataframe for further analysis and potential improvement of prediction accuracy.</p>
After a short view of the descriptor statistics of the capacities of __Supervenue__, it is clear that __Supervenue__ does __not__ indicate venue size or capacity</p>
This was an initial misconception that is stated here for general clarity.

In [ ]:
venues.groupby("Supervenue")[["Max Cap", "U Cap", "Standing Cap"]].mean()

In [ ]:
# Add binary variable Large Venue
venues["Large Venue"] = ((venues["U Cap"] > thresh[0]) | (venues["Standing Cap"] > thresh[1]) | (venues["Max Cap"] > thresh[2]))

In addition, a feature __Area__ is generated from the available numeric features to account for the strong correlation of the numerical values. (see "Relations between variables")</p>
__Assumptions__:
<ul>
    <li>Each seat in the U-Shaped theather accounts for an area with a radius of 2 meters.</li>
    <li>Each standing place accounts for a radius of 0.5 meters.</li>
    <li>Seats in Max Cap have a radius of 2 meter as well. </li>
</ul>
Area = pi * r**2

In [ ]:
import math
# Add area as additional feature
venues["Area"] = venues["U Cap"]*math.pi*2**2 + venues["Standing Cap"]*math.pi*0.5**2 + (venues["Max Cap"] - venues["U Cap"]) * math.pi*1.5**2

Following up on the findings, the difference of the median standing capacities for venues with and without wheelchair ramps is evaluated.</p>
The median is chosen over the mean to mitigate outlier influence.

The median __Standing Cap__ and __Area__ values for venues with and without a wheelchair are given in the table below.</p>

In [ ]:
medians = venues.groupby("WC Ramp")[["Standing Cap", "Area"]].median()
medians

Having a wheelchair ramp seems to be tied to a __high Standing Cap__.</p>

10,000 value sets for __Standing Cap__ were bootstrapped and the difference of medians calculated, as previously described.</p>

In [ ]:
stcap_ci, stcap_p, stcap_bs = bootstrap_diff_metric(
    venues.loc[venues["WC Ramp"] == True, "Standing Cap"], 
    venues.loc[venues["WC Ramp"] == False, "Standing Cap"], 
    np.median, 
    medians.loc[1, "Standing Cap"] - medians.loc[0, "Standing Cap"]
)
stcap_p

The difference __is significant__.  
The __p-values__ is __0.0__ </p>
As the above values are consistent, __we can conclude that large venues, (esp. standing places) are significantly more likely to have a wheelchair ramp than small venues__

### Binary variables

Looking at the __binary variables__, we can draw the following conclusions:
<ul>
  <li>38 % of venues have Loud Music</li>
  <li>74 % of venues serve Alcohol</li>
  <li>93 % of venues provide Wi-Fi</li>
  <li>6 % of venues are Supervenues</li>
    <li>60 % are Promo Events </li>
    <li>6 % are Large Venues </li>
</ul></p>
The distribution for the binary variables is depicted below

In [ ]:
binaries = venues.select_dtypes(include = "bool").drop("WC Ramp", axis = 1)

In [ ]:
# Plot histograms
fig, axs = plt.subplots(1, len(binaries.columns), figsize = (10, 4), sharey = True)
for i, col in enumerate(binaries.columns):
    sns.histplot(binaries[col], ax = axs[i], stat = "probability", binwidth = 0.25)
    axs[i].set_xticks([0.0, 1.0])
    axs[i].set_xticklabels(["False", "True"])
axs[0].set_ylabel("Frequency")
fig.suptitle("The vast majority of venues serves alcohol and provides Wifi. There are only a few supervenues.")
fig.tight_layout()

In [ ]:
# Get frequencies for binary variables
freq = []
for col in binaries.columns:
	freq.append(round(binaries[[col]].value_counts(normalize = True).mul(100),2).astype(str))

## Relations between variables
Next follows the analysis of the relations between the variables, as a preparation for model development. </p>
The conditional probability of a venue having a wheelchair ramp, given that the variable is True is given below.

In [ ]:
# Calculate the ratio of ramps given variable
venues_bool = venues.select_dtypes("bool")
venues_wc_attribute = venues_bool[venues_bool["WC Ramp"] == True].drop("WC Ramp", axis = 1).sum()
total_venues_attribute = venues_bool.drop("WC Ramp", axis = 1).sum()
wc_ratio = round(venues_wc_attribute/total_venues_attribute*100, 2)

In [ ]:
# Calculate the ratio of ramps given the venues does not have the attribute
venues_bool_neg = ~venues.select_dtypes("bool")
venues_wc_attribute_neg = venues_bool_neg[venues_bool_neg["WC Ramp"] == True].drop("WC Ramp", axis = 1).sum()
total_venues_attribute_neg = venues_bool_neg.drop("WC Ramp", axis = 1).sum()
wc_ratio_neg = round(venues_wc_attribute_neg/total_venues_attribute_neg*100, 2)

In [ ]:
conditionals = pd.DataFrame(wc_ratio.append(wc_ratio_neg), columns = ["Ratio [%]"])
conditionals["With Feature"] = ["Yes"]*int(len(conditionals)/2) + ["No"]*int(len(conditionals)/2)
conditionals = conditionals.sort_values(by = "Ratio [%]", ascending = False)

In [ ]:
fig, ax = plt.subplots(figsize = (9, 6))
sns.barplot(y = "Ratio [%]", x = conditionals.index, hue = "With Feature", data = conditionals, ax = ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 45)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title = "Does the venue have the feature?")
plt.xlabel("Features")
plt.ylabel("Frequency [%]")
plt.title("Promo Events have a ramp more often than not \n Supervenues often don't have one")

To check, if the difference of means of installed ramps at supervenues is significant, we used the function from above, receiving a p-value of 0.0.  
Indeed, the average of installed ramps at supervenues is significantly smaller than at normal venues.

In [ ]:
mean_diff = venues.loc[venues["Supervenue"] == False, "WC Ramp"].sum() - venues.loc[venues["Supervenue"] == True, "WC Ramp"].sum()
super_ci, super_p, super_bx = bootstrap_diff_metric(
    venues.loc[venues["Supervenue"] == False, "WC Ramp"],
    venues.loc[venues["Supervenue"] == True, "WC Ramp"],
    np.mean,
    mean_diff,)

From this we can already isolate two important variables:
_supervenue_ is an indicator for __not__ having a wheelchair ramp: P(ramp | Supervenue) = 30.29 % </p>
_Promo Events_ have a wheelchair ramp more often than not: P(ramp | Promo Event) = 60.64 % </p>
Venues without Wi-Fi also seem to have a ramp more often.</p>
For the remaining variables, there is a ramp approx. as often as not, given they are True.

In [ ]:
# Redefine a df numerics
numerics = venues.select_dtypes(include = "number")
numerics["WC Ramp"] = venues["WC Ramp"]

The correlation matrix for venues is visualized as a heatmap. </p>
The dataset contains binary and numerical variables with strong outliers. The Pearson correlation is not the best metric for this, because:</p>
<ul>
  <li>Limited to continuous variables</li>
  <li>Only accounts for linear relationship</li>
  <li>Sensitive to outliers</li>
</ul></p>
Instead, the __phi_k coefficient__ is chosen as a metric to measure the correlation between the variables. </p>
(cp: <href>https://arxiv.org/abs/1811.11440</href>)

In [ ]:
# Print the heatmap of the phi_k correlation of the variables
# means.columns is taken from the pen-ultimate code-cell to specify the interval_cols
fig, axs = plt.subplots(1, 2, figsize = (12, 6))
venues = venues.reindex((sorted(venues.drop("WC Ramp", axis = 1).columns, reverse = True)).insert(0, "WC Ramp"), axis=1)
sns.heatmap(round(venues.drop("Venue", axis = 1).phik_matrix(interval_cols = venues.select_dtypes("number").columns), 2), annot = True, ax = axs[0])
axs[0].set_title("Correlation Matrix (phi_k)")

significance_overview = venues.drop("Venue", axis = 1).significance_matrix(interval_cols=venues.select_dtypes("number").columns)
sns.heatmap(round(significance_overview[["WC Ramp"]].drop("WC Ramp", axis = 0), 1), annot = True, vmin = -5, vmax = 5, square = "equal", cmap = "seismic", ax = axs[1])
axs[1].set_title("Significance of Coefficients")
plt.tight_layout()

The left plot shows the Correlation matrix. </p> In addition to that, the significance of the correlation coefficients with the dependent variable "WC Ramp" is plotted to the right. The right colormap saturates at +/- 5 standard deviations.</p>
We can verify that especially the __correlation__ with __Large Venue__, __Promo Events__ and __Area__ is __significant__.</p>
Except for "Loud Music" and "Wi-Fi", all values are > 5*std</p>

The variables that are "strongest" correlated to __WC Ramp__ are:  
<ol>
  <li>Large Venue: 0.27 (+)</li>
  <li>Promo Events: 0.21 (+)</li>
  <li>Area: 0.17 (+)</li>
  <li>Supervenue: 0.17 (-)</li>
  <li>Alcohol: 0.16 (+)</li>
</ol></p>

Further remarks:</p>
<ul>
    <li>Promo Events and Loud Music are strongly correlated: 0.5</li>
    <li>Loud Music and Alcohol are correlated: 0.37</li>
    <li>Max Cap and Standing Cap are strongly correlated: 0.89</li>
</ul>

It is to note, that the __correlations with WC Ramp__ are __weak__.</p>
The phi_k coefficient does not indicate direction, as the Pearson correlation does.  
The direction given above is inferred from the previous analysis.</p>

Furthermore, the created variable __Area__ seems to change with __Loud Music__ and __Alcohol__.

An additional column is created:  
__Party__ := True, iff Loud Music == True or Promo Events == True or Alcohol == True </p>
The correlation with WC Ramp is __0.2152__

In [ ]:
venues["Party"] = venues["Promo Events"] | venues["Loud Music"] | venues["Alcohol"]

In [ ]:
round(venues[["WC Ramp", "Party"]].phik_matrix(), 2)

# Model Development
The goal is to predict if a given venue already has a ramp. </p>
Minimum requirements:  
True-Negative-Rate > 2/3  
High Recall

## Methods  
First, a vanilla __Random Forest Classifier__, a __Gradient Boost Classifier__, __K-Nearest Neighbors Classifer__ and a __Support Vector Classifier__ are fit to the dataset. If possible, all have random_state = 3078.   The model search space is limited to shallow algorithms, as they are generally better suited for tabular data than artificial neural networks (resource-intensive, bad interpretability)</p>
The numeric features are scaled with sklearn's __StandardScaler()__. (This is not necessary for the ensemble / tree methods, although sometimes beneficial.) </p>
All independent features (variables) are used for this.</p>
__Data-Split__: The dataset is split up __0.8 : 0.2__ (train:test) </p>
__Feature importance__ is measured. We derive the most important features to decrease dimensionality. </p>
__Hyperparameter optimization__ is executed. The __Optuna__ library is used for this. 100 trials are executed. (see Appendix)</p>
The best model params are retrieved and used to evaluate scores on a cross_validated dataset. </p>
The reported metrics are: __confusion matrix__, __accuracy__, __precision__, __recall__, __f1-score__. Accuracy is taken from 5-fold stratified cross-validation on the 80 % training data. </p>
The final model is trained on only 5 features on the 80 % training dataset, with competitive results to the model trained on the entire dataset.</p>

(Preliminary experiments with an XGBoostClassifier were executed. This didn't render any improvement to the sklearn classifiers and was thus omitted)

In [ ]:
# Necessary imports
import sklearn
from typing import Any
import copy
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import classification_report, precision_score, average_precision_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [ ]:
venues = venues.set_index("Venue")

In [ ]:
# confusion_matrix_scorer from sklearn-website adapted:
def confusion_matrix_scorer(clf, X, y):
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred)
    return {'tn': cm[0, 0], 'fp': cm[0, 1],'fn': cm[1, 0], 'tp': cm[1, 1]}

# Define training function
def train_classifier(clf_params_dict: dict,
                     X: pd.DataFrame,
                     y: pd.DataFrame,
                     scaler: Any = None,
                     train_size: float = 0.8,
                    ) -> dict:
    """ 
    This function trains the given classifiers.
    Args:
        clf_params_dict: dict: with signature: {"clf_name":{"clf":sklearn_model, "params":dict(params)}
        X: pd.DataFrame: the training data
        y: pd.DataFrame: the labels
        scaler: Any (optional): a sklearn scaler object
        train_size: float (Default=0.8): the train_size for train_test_split
    Returns:
        output: Dict: a dict with signature 
                    {"clf_name":{"conf_matrix":confusion_matrix, "accuracy":accuracy, "model":trained model}}
    """   
    # Split dataset in train_test
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size)
    
    if scaler is not None:
        X_index = X_train.select_dtypes("number").columns
        X_train[X_index] = scaler.fit_transform(X_train[X_index])
        X_test[X_index] = scaler.transform(X_test[X_index])
    
    # Train classifiers and store model, accuracy and confusion matrix in dict output
    output = {}
    for k, v in clf_params_dict.items():
        clf = v["clf"]
        if v["params"] is not None:
            clf.set_params(**v["params"])
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        
        # Note: cross_validate is used three times here. For faster execution this can be strongly optimized.
        score = cross_val_score(clf, X_train, y_train, n_jobs = -1, cv = 5, scoring = "f1")
        accuracy = score.mean()
        accuracy = (accuracy_score(y_test, preds) + accuracy)/2

        score_conf = cross_validate(clf, X_train, y_train, n_jobs = -1, cv = 5, scoring = confusion_matrix_scorer)
        matrix = np.array([0, 0, 0, 0])
        matrix[0] = score_conf["test_tn"].mean()
        matrix[1] = score_conf["test_fp"].mean()
        matrix[2] = score_conf["test_fn"].mean()
        matrix[3] = score_conf["test_tp"].mean()
        matrix = matrix.reshape(2,2)
        conf_matrix = matrix/matrix.sum(axis = 1)
        
        scoring = {"precision":"average_precision", "recall":"recall", "f1":"f1"}
        metrics = cross_validate(clf, X_train, y_train, n_jobs = -1, cv = 5, scoring = {"precision":"average_precision", "recall":"recall", "f1":"f1"})
        
        output[k] = {"conf_matrix":0, "accuracy":0, "model":0}
        output[k]["conf_matrix"] = conf_matrix
        output[k]["accuracy"] = accuracy
        output[k]["model"] = clf
        output[k]["metrics"] = metrics
        
    return output    

In [ ]:
# Define feature importances
def plot_feature_importances(
    clf_dict: dict,
    feat_import: pd.DataFrame,
) -> Any:
    """ 
    This function plots the feature importances of sklearn models, that have the attribute
    feature_importances_
    Args:
        clf_dict: a dictionary with keys: classifier names, values: trained classifiers
        feat_import: a DataFrame with the Feature Names in the first column
    Returns:
        A plot of the feature importances
    """
    fig, ax = plt.subplots(figsize = (6, 5))
    for k, v in clf_dict.items():
        feat_import[k] = pd.Series(v.feature_importances_)

    feat_import_melt = pd.melt(
        feat_import, 
        id_vars = "Feature", 
        value_vars = clf_dict.keys(), 
        value_name = "Importances",
        var_name = "Classifier"
    )
    sns.barplot(
        x = "Importances", 
        y = "Feature", 
        hue = "Classifier", 
        data = feat_import_melt.sort_values("Importances", ascending = False),
        orient = "h",
        ax = ax)
    most_important = feat_import_melt[
        feat_import_melt["Importances"] == feat_import_melt["Importances"].max()
    ]["Feature"].to_numpy()
    
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)
    ax.set_ylabel("Importance")
    fig.suptitle(f"The feature '{most_important.squeeze()}' Has Highest Influence on the Prediction")
    fig.tight_layout()

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(
    conf_matrix_dict: dict,
    title: str = None,
) -> Any:
    """ 
    This function plots the confusion matrices as heatmaps
    Args:
        conf_matrix_dict: a dictionary with keys: classifier names, values: confusion matrices
        title: An optional plot title
    Returns:
    	A plot of the confusion matrices
    """
    fig, axs = plt.subplots(1, len(conf_matrix_dict), sharey = True, figsize = (10, 4))
    i = 0
    for k, v in conf_matrix_dict.items():
        if i < len(conf_matrix_dict) - 1:
        	sns.heatmap(v, ax = axs[i], annot = True, xticklabels = ["Negative", "Positive"], cbar = False)
        else:
            sns.heatmap(v, ax = axs[i], annot = True, xticklabels = ["Negative", "Positive"], cbar = True)
        axs[i].set_title(k)
        axs[i].set_xlabel("Prediction")
        i += 1
        axs[0].set_yticklabels(["Negative", "Positive"])
        if title is None:
        	fig.suptitle("Confusion Matrices")
        else:
            fig.suptitle(title)
    axs[0].set_ylabel("Ground Truth")
    fig.tight_layout()

In [ ]:
###############################################################################
# This might take a while!

# Instantiate StandardScaler for numerical values (not obligatory for tree-like, but might be beneficial)
scaler = StandardScaler()

# Create dataset
y = venues["WC Ramp"]
X_cols = venues.drop("WC Ramp", axis =1).columns
X = venues[X_cols]

# Instantiate models
forest_clf = RandomForestClassifier(random_state = 3078)
gb_clf = GradientBoostingClassifier(random_state = 3078)
knn = KNeighborsClassifier()
svc = SVC(random_state = 3078)

# Define dict
clf_params_dict = {"KNN":{"clf": knn, "params":None},
                   "GBC":{"clf": gb_clf, "params":None},
                   "Forest":{"clf": forest_clf, "params":None},
                   "SVC":{"clf": svc, "params":None},
                  }

# Train models
clf_dict = train_classifier(clf_params_dict, X, y, scaler = StandardScaler())

## Feature Selection
The feature importances are plotted below for the tree-based Forest and GradBoost Classifiers.</p>
We can see that the feature engineering for __Party__ and __Area__ seems successful.  
__Area__ is among the top three most important features.  
__Party__ is among the most important binary features for GradBoost.  
In general, the numeric features are more important than the binary features.

In [ ]:
# Plot feature importances
clf_dict_fi = {"GradBoost":clf_dict["GBC"]["model"], "Forest":clf_dict["Forest"]["model"]}
feat_import = pd.DataFrame(X.columns, columns = ["Feature"])

plot_feature_importances(clf_dict_fi, feat_import)

Next the importance of each feature was analyzed, using PCA. </p>
(The code snippet below was gratefully adapted from this great blog post: https://towardsdatascience.com/3-essential-ways-to-calculate-feature-importance-in-python-2f9149592155)

In [ ]:
# Perform PCA
scaler = StandardScaler()
X_scaled = pd.DataFrame(X)
X_scaled[X.select_dtypes("number").columns] = scaler.fit_transform(X.select_dtypes("number"))
pca = PCA().fit(X_scaled)

# Compute the loadings
loadings = pd.DataFrame(
    data=pca.components_.T * np.sqrt(pca.explained_variance_), 
    columns=[f'PC{i}' for i in range(1, len(X_scaled.columns) + 1)],
    index=X_scaled.columns
)
loadings.head()

In [ ]:
# Visualize the loadings
pc1_loadings = loadings.sort_values(by="PC1", ascending=False)[["PC1"]]
pc1_loadings = pc1_loadings.reset_index()
pc1_loadings.columns = ["Attribute", "CorrelationWithPC1"]

fig, axs = plt.subplots(1, 2, figsize = (10, 5))
axs[1].bar(x=pc1_loadings["Attribute"], height=pc1_loadings["CorrelationWithPC1"])
axs[1].set_title("PCA loading scores (first principal component)", size=11)
axs[1].set_ylabel("Loading Score")
axs[1].set_xlabel("Feature")
plt.xticks(rotation="vertical")

axs[0].plot(pca.explained_variance_ratio_.cumsum(), lw=3)
axs[0].set_title("3 components explain 90 % of variance", size=11)
axs[0].set_ylabel("Cum. explained variance")
axs[0].set_xlabel("Number of components")
fig.tight_layout()

The above plots confirm the intuition from the feature importance plot.</p>
The left plot shows that only 3 components explain 90 % of the variance in the data.</p>
The plot to the right shows the correlation of each feature with the first principal component.<p>

## Accuracy and Confusion

The confusion matrices for the respective classifiers are visualized below.</p>
This provides valuable insight. __GradBoost__ seems to have the __most balanced__ performance with regards to False-Positives and False-Negatives and the best prediction accuracy.

In [ ]:
# Plot confusion matrix for initial classifiers
conf_matrix_dict = {
    "KNN":clf_dict["KNN"]["conf_matrix"], 
    "GradBoost":clf_dict["GBC"]["conf_matrix"], 
    "Forest":clf_dict["Forest"]["conf_matrix"], 
    "SVC": clf_dict["SVC"]["conf_matrix"]
}

plot_confusion_matrix(conf_matrix_dict, "Comparison of Initial Classifiers")

The total accuracy varies between ~48 - 64 % on a 5-fold cross-validation of the training set.

In [ ]:
# Plot accuracy for initial classifiers
accuracies = {k:v["accuracy"] for k, v in clf_dict.items()}
acc = pd.DataFrame.from_dict(accuracies, orient = "index", columns = ["Accuracy [%]"])
ax = sns.barplot(x = acc.index, y = acc.loc[:,"Accuracy [%]"], data = acc)
ax.set_title("The max accuracy is around 64 %", fontsize = 15)

A hyperparameter search was executed, using Optuna. The code is given in the Appendix for reproducibility. </p>
Based on the feature analysis, __Max Cap__, __Standing Cap__, __Area__, __U Cap__ and __Promo Events__ are selected going forward. The results of the vanilla classifiers on these features are shown below. Of the numeric features only U-Cap has been omitted.</p>

In [ ]:
###############################################################################
# This might take a while!

# Instantiate StandardScaler for numerical values (not obligatory for boosting tree, but might be beneficial)
scaler = StandardScaler()

# Create dataset
y = venues["WC Ramp"]
X_cols = ["U Cap", "Promo Events", "Area", "Max Cap", "Standing Cap"]
X = venues[X_cols]

# Train models
clf_dict_red = train_classifier(clf_params_dict, X, y, scaler = StandardScaler())

In [ ]:
conf_matrix_dict_red = {
    "KNN":clf_dict_red["KNN"]["conf_matrix"], 
    "GradBoost":clf_dict_red["GBC"]["conf_matrix"], 
    "Forest":clf_dict_red["Forest"]["conf_matrix"], 
    "SVC": clf_dict_red["SVC"]["conf_matrix"]
}

plot_confusion_matrix(conf_matrix_dict_red, "Grad Boost Shows Best Result on the Reduced Dataset")

The results are very similar to the initial classifiers, but with only 50 % of the features.</p>
This can be a helpful approach to reduce resources and save time for larger datasets and shows that the feature selection was successful.

In [ ]:
# Train best classifier
optim_params = {"max_depth": 3, 'n_estimators': 98, 'min_samples_split': 2, "random_state" : 3078}
#{'max_depth': 5, 'n_estimators': 49, 'min_samples_split': 3, "random_state":3078}
best_clf_dict = train_classifier({"GBC":{"clf": GradientBoostingClassifier(), "params":optim_params}}, X, y, scaler = StandardScaler())

In [ ]:
conf_matrix_dict_red = {
    "GradBoost Final":best_clf_dict["GBC"]["conf_matrix"],
    "GradBoost Red":clf_dict_red["GBC"]["conf_matrix"],
    "GradBoost Initial":clf_dict["GBC"]["conf_matrix"],
}

plot_confusion_matrix(conf_matrix_dict_red, "Confusion Matrices for the Different Stages of the Model")

The feature selection drove the model to a __higher true-negative rate__</p>
However, the initial model has the most balanced prediction accuracy, with respect to true-negatives and true-positives.</p>
The selection of the right model depends on the emphasis being put on either predicting venues without ramps, or with ramps. </p>
In general, all three models reach the goal of a True-Negative Rate of > 2/3

The final accuracies can be seen below.</p>
__GradBoostFinal__ and __GradBoost Red__ operate on only __50 % of the features__.  
__GradBoostFinal__ is __hyperparameter-tuned__.  
__GradBoost Initial__ was trained on __all features__ and with __vanilla hyperparameters__.

In [ ]:
clf_acc_dict = {
    "GradBoost Final":best_clf_dict["GBC"]["accuracy"],
    "GradBoost Red":clf_dict_red["GBC"]["accuracy"],
    "GradBoost Initial":clf_dict["GBC"]["accuracy"],
}
accuracies = {k:v for k, v in clf_acc_dict.items()}
acc = pd.DataFrame.from_dict(accuracies, orient = "index", columns = ["Accuracy"])
ax = sns.barplot(x = acc.index, y = acc.loc[:,"Accuracy"], data = acc)
ax.set_title("Comparison of the Cross-Val Accuracies")
print(acc)

In [ ]:
clf_metrics_dict = {"GradBoost Final":best_clf_dict["GBC"]["metrics"],
    "GradBoost Red":clf_dict_red["GBC"]["metrics"],
    "GradBoost Initial":clf_dict["GBC"]["metrics"],
    }
precisions = {k:v["test_precision"].mean() for k, v in clf_metrics_dict.items()}
recalls = {k:v["test_recall"].mean() for k, v in clf_metrics_dict.items()}
f1s = {k:v["test_f1"].mean() for k, v in clf_metrics_dict.items()}

In [ ]:
metrics = pd.DataFrame()
metrics = metrics.append(recalls, ignore_index = True).append(precisions, ignore_index = True).append(f1s, ignore_index = True)
metrics["Metric"] = ["Precision","Recall","F1-Score"]
metrics = pd.melt(metrics, id_vars = "Metric", value_vars = metrics.drop("Metric", axis = 1).columns, value_name = "Result", var_name = "Classifier").sort_values("Result")

In [ ]:
fig, ax = plt.subplots(figsize = (8, 5))
sns.scatterplot(y = "Result", x = "Metric", hue = "Classifier", data = metrics, ax = ax)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title = "Classifier")
plt.title("Performance on entire dataset ~ selected dataset \n Tuning can improve the model. (x-lim = [0.4, 0.8])", fontsize = 15, pad = 10)
ax.set_ylim([0.25, 0.8])
fig.tight_layout()

## ML Explainer
Finally, we want to visualize the influence of our features on the prediction probability of our model.</p>
LIME is used as a library to accomplish this (note: SHAP was not working, because of some dependency conflicts with the installed numpy version == 1.22.3)</p>
(It might be that this has to be viewed in the Jupyter Notebook, as the new workspace editor did not allow me to visualize .html.)

In [ ]:
import lime
from lime import lime_tabular

# Prepare LIME explainer
scaler = StandardScaler()
X_scaled_red = pd.DataFrame(X)
X_scaled_red[X_scaled_red.select_dtypes("number").columns] = scaler.fit_transform(X_scaled_red.select_dtypes("number"))

explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X),
    feature_names=X.columns,
    class_names=['False', 'True'],
    mode='classification'
)

The ML-Explainer shows us the predicted probabilities for a chosen observation -> the observation can be changed by setting the value of _EXAMPLE_. </p>
This is generally in line with our previous findings:</p>
__High Max Cap__, __Area__ and __Standing Cap__ usually are __good predictors__ for an existing ramp. </p>
Further, __Promo Events__ is a __weak predictor__ for having a ramp as well.

In [ ]:
# Plot lime explainer for EXAMPLE
# This value can be adjusted to vary the example that is visualized
EXAMPLE = 150
exp = explainer.explain_instance(
    data_row=X.iloc[EXAMPLE], 
    predict_fn=best_clf_dict["GBC"]["model"].predict_proba
)

exp.save_to_file("explained.html")
exp.show_in_notebook(show_table=True)

# Conclusion

A dataset, containing the mapping of event venues and their attributes to whether they have a wheelchair ramp or not, was analyzed.</p>
Background was the pursuit of the Marketing team to expand the business to event venues.</p>
For this, a model with a true-negative-rate of at least 2/3 and a test-accuracy of > 60% was developed. </p>
</p>
The most important features were identified (+/-: relation):
<ul>
  <li>Maximum Capacity (+)</li>
  <li>Standing Capacity (+)</li>
  <li>Promo Events (+)</li>
  <li>Estimated Venue Area (+)</li>
  <li>U-Shape Capacity (+)</li>
</ul></p>
It can be summarized that large venues are equipped with wheelchair ramps significantly more often than small venues. This is due to the fact that venues with a high standing capacity have a high number of ramps. </p>
Only ~30% of supervenues do have a wheelchair ramp. </p>
Party venues are usually also already accessible via wheelchair. </p>

According to our findings, __the recommendation is, to target small- & mid-sized venues, with few standing capacities, as well as high-quality venus (supervenues).__ For the following pre-selection and search for venues, the following thresholds are given:
<ul>
    <li>U-Shape Capacity: max. 200 </li>
    <li>Standing Capacity: max. 500 </li>
    <li>Maximum Theater Capacity: max. 200</li>
</ul>

A corresponding model was fitted and can predict if a venue has a ramp, or not with an accuracy of ~65%.</p>
This can help with a pre-selection. Further information, such as exact location of all venues, as well as real areas, could help to improve the model.</p>
The model can be adjusted in a certain range to either have a higher true-negative, or true-positive-rate, depending on the emphasis of the Marketing Team.</p>
Since model accuracy is still in the lower range, further investigation and data collection should be the next step to improve performance.</p>

# Appendix

## Hyperparameter optimization

In [ ]:
USE_HYPEROPT = False

In [ ]:
if USE_HYPEROPT:
    import sklearn

    import optuna

    def confusion_matrix_scorer(clf, X, y):
            y_pred = clf.predict(X)
            cm = confusion_matrix(y, y_pred)
            return {'tn': cm[0, 0], 'fp': cm[0, 1],
                    'fn': cm[1, 0], 'tp': cm[1, 1]}

    # Hyperparameter optimization
    def objective(trial):

        # Suggest values for the hyperparameters using a trial object.
        classifier_name = trial.suggest_categorical("classifier", ["SVC", "Forest", "GradBoost"])
        if classifier_name == "SVC":
             svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
             classifier_obj = SVC(C=svc_c, gamma='auto', random_state = 3078)

        elif classifier_name == "Forest":
            rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
            rf_estimators = trial.suggest_int("rf_estimators", 10, 1e4, log = True)

            classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_max_depth, random_state = 3078)

        elif classifier_name == "GradBoost":
            gb_max_depth = trial.suggest_int("gb_max_depth", 3, 100, log = True)
            gb_estimators = trial.suggest_int("gb_estimators", 1, 100, log = True)
            gb_min_split = trial.suggest_int("gb_min_split", 2, 10, log = False)

            classifier_obj = GradientBoostingClassifier(max_depth=gb_max_depth, n_estimators=gb_estimators, min_samples_split=gb_min_split, random_state = 3078)

        else:
            optimizer = trial.suggest_categorical('algorithm', ['auto','ball_tree','kd_tree','brute'])
            knn_neighbors = trial.suggest_int("knn_neighbors", 2, 1e2, log=True)
            knn_leafs = trial.suggest_int("knn_leafs", 5, 100, log = True)

            knn = KNeighborsClassifier(n_neighbors=knn_neighbors,algorithm=optimizer, leaf_size = knn_leafs)
        score = cross_val_score(classifier_obj, X, y, n_jobs=-1, cv=5, scoring = "accuracy")
        accuracy = score.mean()
        return accuracy

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)
    trial_best = study.best_trial
    trial_best

## END OF REPORT